# First Agent
We'll use smolagents, a library that provides a freamework for developing your agents with ease. not suitable for production grade

## What does this agent do?

In [1]:
from smolagents import CodeAgent, tool, InferenceClientModel
import os
from dotenv import load_dotenv

/Users/elenali/jupyter-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [34]:
# LLM model
model_id = 'Qwen/Qwen3-4B-Instruct-2507'
model = InferenceClientModel(
    max_tokens = 2096,
    model_id = model_id,
)

In [4]:
# simply use the model to generate response, not an agent here
agent = CodeAgent(model= model, tools = [])
result = agent.run("I need your help.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need your help.                                                                                               │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print("Please provide the specific task or question you need help with.")                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Please provide the specific task or question you need help with.

Out: None

[Step 1: Duration 1.61 seconds| Input tokens: 1,941 | Output tokens: 48]

...

[Step 3: Duration 1.69 seconds| Input tokens: 6,390 | Output tokens: 292]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68b776a5-4e993bf72a8d483b3172c079;f43df722-7951-41db-b324-056565756daf)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 4: Duration 0.26 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68b776a5-4e993bf72a8d483b3172c079;f43df722-7951-41db-b324-056565756daf)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [6]:
# define tools
@tool
def my_custom_tool (arg:str) -> str:
    """A dummy tool that does nothing
    Args:
        arg: dummy arg
    """
    return "Here is your magic tool that does nothing. "

In [ ]:
The CodeAgent assumes complex tasks might require multiple steps of reasoning and code execution:

Step 1: Analyze the problem and write initial code
Step 2: Examine results and decide if more work is needed
Step 3: Write additional code if necessary
Continue: Until the task is "complete"

In [13]:
# Agent with dummy tool
from smolagents.agents import EMPTY_PROMPT_TEMPLATES
prompt_templates = EMPTY_PROMPT_TEMPLATES.copy()

prompt_templates['system_prompt'] = """
        You are an assistant with access to a special tool called `my_custom_tool`.
        Whenever the user asks to "see magic", "show magic", "do magic", or otherwise refers
        to something magical, you MUST call `my_custom_tool` with the user’s provided phrase
        as the `arg` argument. 

        Format your reasoning as follows:
        
        Thoughts: I need to call the magic tool.
        <code>
        # Your python code here
        my_custom_tool(arg="the magic tool")
        </code>
        Observation: "Here is your magic tool that does nothing. "
        
        - If the user does not provide an argument, pass "magic" as the default.
        - Do not try to answer the request directly when it involves magic—always use the tool.
        - After calling the tool, return its output to the user.
        - For all other requests not related to magic, just answer normally without calling the tool.
        """ 

agent_dummy = CodeAgent(
    model= model, 
    tools = [my_custom_tool],
    prompt_templates = prompt_templates,
    max_steps = 1
)
result = agent_dummy.run("Show me some magic!")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Show me some magic!                                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-4B-Instruct-2507 ────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  my_custom_tool(arg="Show me some magic!")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: Here is your magic tool that does nothing.

[Step 1: Duration 1.45 seconds| Input tokens: 219 | Output tokens: 27]

Reached max steps.

[Step 2: Duration 2.21 seconds| Input tokens: 344 | Output tokens: 189]

In [8]:
from datetime import datetime
import time

@tool
def get_time()->str:
    """A tool that fetches the current local time."""
    try:
        now = datetime.now()
        return f"🕒 It's {now.strftime('%I:%M %p')} on {now.strftime('%A, %B %d')}"
    except Exception as e:
        return f"❌ Error getting time: {str(e)}"
        
    
    

The CodeAgent assumes complex tasks might require multiple steps of reasoning and code execution:

Step 1: Analyze the problem and write initial code
Step 2: Examine results and decide if more work is needed
Step 3: Write additional code if necessary
Continue: Until the task is "complete"

In smolagents, the FinalAnswerTool is a special built-in tool that lets an agent end its reasoning loop and return the final answer to the user.

Think of it as the "exit door" for the agent:

During a reasoning chain, the agent might call tools, write Python, or fetch external info.

Once it has the actual result it wants to deliver, it must call FinalAnswerTool with that answer.

This prevents the agent from looping forever and makes it explicit: “I’m done, here’s the output.”
Why it matters

Without FinalAnswerTool, the agent might just keep reasoning and calling tools until it hits max_steps. By calling FinalAnswerTool, it signals to the framework that it has produced the answer for the user.

In [14]:
from smolagents import FinalAnswerTool
final_answer = FinalAnswerTool()
prompt_templates = EMPTY_PROMPT_TEMPLATES.copy()

prompt_templates['system_prompt'] = """
    

    """

agent_time = CodeAgent(
    model= model, 
    tools = [get_time, final_answer],
    max_steps = 2,
    verbosity_level = 2,
    prompt_templates = prompt_templates
    
)
result = agent_time.run("What time?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What time?                                                                                                      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-4B-Instruct-2507 ────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I can't provide the current time directly, as this would require real-time access that I don't have. However, you  
can check the current time on your device's clock, or you can tell me your location and time zone, and I can assist
you further! 😊                                                                                                    

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
Here is your code snippet:
I can't provide the current time directly, as this would require real-time access that I don't have. However, you 
can check the current time on your device's clock, or you can tell me your location and time zone, and I can assist
you further! 😊</code>
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
<code>
# Your python code here
</code>
Make sure to provide correct code blobs.

[Step 1: Duration 1.31 seconds| Input tokens: 20 | Output tokens: 57]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # No code needed — this query cannot be answered with real-time data.                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: None

[Step 2: Duration 1.31 seconds| Input tokens: 270 | Output tokens: 146]

Reached max steps.

[Step 3: Duration 0.24 seconds]

In [ ]:
The parameter verbosity_level controls how much the agent logs or “thinks out loud” during its reasoning and tool use.

Levels

verbosity_level=0 → Silent/minimal logging

Only the final result is returned.

No intermediate reasoning, tool calls, or debug info shown.

verbosity_level=1 → Normal / concise logging (default)

Prints the agent’s steps (like Thoughts: ..., tool calls, and observations), but in a summarized way.

Good balance if you want to see what’s happening without too much clutter.

verbosity_level=2 → Verbose / full trace

Shows detailed chain-of-thought (well, the structured reasoning it’s allowed to show), intermediate code snippets, tool arguments, and outputs.

Useful for debugging or when developing custom tools.

In [ ]:
some built-in tools
https://huggingface.co/docs/smolagents/reference/default_tools

# Observability


In [28]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
from langfuse import get_client
from langfuse import Langfuse

LANGFUSE_SECRET_KEY=os.getenv("LANGFUSE_SECRET_KEY"),
LANGFUSE_PUBLIC_KEY=os.getenv("LANGFUSE_PUBLIC_KEY"),
LANGFUSE_HOST=os.getenv("LANGFUSE_HOST")


In [29]:
# Verify connection
langfuse = get_client()
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


In [30]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

SmolagentsInstrumentor().instrument()


Attempting to instrument while already instrumented


AttributeError: 'Langfuse' object has no attribute 'trace'

In [40]:
query = "How can I know the time?"
result = agent_time.run(query)

TypeError: Langfuse.start_as_current_span() got an unexpected keyword argument 'tags'